# Movies Database

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_table(r"datasets/movie datasets/imdb/title.basics.tsv/data.tsv", sep="\t")

C:\Users\smend\AppData\Local\Temp\ipykernel_16004\1166172987.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_table(r"datasets/movie datasets/imdb/title.basics.tsv/data.tsv", sep="\t")


In [ ]:
df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


# Analysing dataset

In [6]:
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
from functools import reduce
from transformers import AutoTokenizer
import regex as re
from palm_process import generate_questions_palm2
tqdm.pandas()

In [7]:
def extract_additional_information(html):
    soup = BeautifulSoup(html)
    synopsis = soup.find('span', id='synopsis')
    if synopsis:
        synopsis_text = synopsis.find("div", {"class": "ipc-html-content-inner-div"})
        if synopsis_text:
            return synopsis_text.text
    summaries = [div.text for div in soup.find_all("div", {"class": "ipc-html-content-inner-div"}) if div]
    summaries = sorted(summaries, key=len, reverse=True)
    return summaries[0] if summaries else ""

In [8]:
def clean_string(string):
    string_list = string.split("—")
    if len(string_list)>1:
       string =  " ".join(string_list[:-1])
    return re.sub("\s+", " ", string)

In [9]:
tokenizer = AutoTokenizer.from_pretrained("openlm-research/open_llama_3b")

In [5]:
from glob import glob

In [12]:
filelist = glob("datasets/movie_datasets/imdb/*_movies_html_subset.feather")
print(filelist)

['datasets/movie_datasets/imdb\\40_movies_html_subset.feather', 'datasets/movie_datasets/imdb\\41_movies_html_subset.feather', 'datasets/movie_datasets/imdb\\42_movies_html_subset.feather', 'datasets/movie_datasets/imdb\\43_movies_html_subset.feather', 'datasets/movie_datasets/imdb\\44_movies_html_subset.feather', 'datasets/movie_datasets/imdb\\45_movies_html_subset.feather', 'datasets/movie_datasets/imdb\\46_movies_html_subset.feather', 'datasets/movie_datasets/imdb\\47_movies_html_subset.feather', 'datasets/movie_datasets/imdb\\48_movies_html_subset.feather', 'datasets/movie_datasets/imdb\\49_movies_html_subset.feather', 'datasets/movie_datasets/imdb\\50_movies_html_subset.feather', 'datasets/movie_datasets/imdb\\51_movies_html_subset.feather', 'datasets/movie_datasets/imdb\\52_movies_html_subset.feather', 'datasets/movie_datasets/imdb\\53_movies_html_subset.feather', 'datasets/movie_datasets/imdb\\54_movies_html_subset.feather', 'datasets/movie_datasets/imdb\\55_movies_html_subset.f

In [13]:
# df = reduce(lambda x,y: pd.concat([x, y], axis=1), map(pd.read_feather, filelist))
df = pd.read_feather(filelist[0])

In [14]:
df.head()

,tconst,originalTitle,startYear,genres,runtimeMinutes,averageRating,numVotes,main_html,plot_html
20000,tt6398990,Karaoke Knights,2018,Comedy,\N,7.6,9.0,"<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...","<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht..."
20001,tt6398994,Failing Grace,2018,Thriller,81,5.6,11.0,"<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...","<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht..."
20002,tt6399092,Enormous: The Gorge Story,2019,"Documentary,History,Music",65,8.0,74.0,"<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...","<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht..."
20003,tt6399296,Inmate #1: The Rise of Danny Trejo,2019,"Biography,Documentary",107,7.7,1248.0,"<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...","<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht..."
20004,tt6399814,Freak in the Basement,2018,Horror,49,2.1,39.0,"<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...","<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht..."


In [15]:
df["synopsis"] = df["plot_html"].progress_apply(extract_additional_information)

100%|██████████| 500/500 [00:22<00:00, 22.51it/s]


In [16]:
df["synopsis"] = df["synopsis"].progress_apply(clean_string)

100%|██████████| 500/500 [00:00<00:00, 15624.15it/s]


In [17]:
df.head()

,tconst,originalTitle,startYear,genres,runtimeMinutes,averageRating,numVotes,main_html,plot_html,synopsis
20000,tt6398990,Karaoke Knights,2018,Comedy,\N,7.6,9.0,"<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...","<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...",After a die-hard karaoke singer dies hard at a...
20001,tt6398994,Failing Grace,2018,Thriller,81,5.6,11.0,"<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...","<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...","Five years ago, sisters Maggie (Curtis) and Gr..."
20002,tt6399092,Enormous: The Gorge Story,2019,"Documentary,History,Music",65,8.0,74.0,"<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...","<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...","Despite overwhelming odds, a small family wine..."
20003,tt6399296,Inmate #1: The Rise of Danny Trejo,2019,"Biography,Documentary",107,7.7,1248.0,"<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...","<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...","71 years in the making, Inmate #1: The Rise of..."
20004,tt6399814,Freak in the Basement,2018,Horror,49,2.1,39.0,"<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...","<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...",Nicolai Sctulman has got a big secret - in his...


In [18]:
df["synopsis_len"] = df["synopsis"].progress_apply(lambda x: tokenizer(x)["input_ids"]).apply(len)

100%|██████████| 500/500 [00:00<00:00, 1486.52it/s]


In [19]:
df["synopsis_len"].describe()

count     500.000000
mean      200.548000
std       482.553071
min         1.000000
25%        34.750000
50%        70.500000
75%       147.000000
max      6033.000000
Name: synopsis_len, dtype: float64

In [20]:
df[(df["synopsis_len"]>75) & (df["synopsis_len"]<150)]

,tconst,originalTitle,startYear,genres,runtimeMinutes,averageRating,numVotes,main_html,plot_html,synopsis,synopsis_len
20002,tt6399092,Enormous: The Gorge Story,2019,"Documentary,History,Music",65,8.0,74.0,"<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...","<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...","Despite overwhelming odds, a small family wine...",83
20004,tt6399814,Freak in the Basement,2018,Horror,49,2.1,39.0,"<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...","<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...",Nicolai Sctulman has got a big secret - in his...,80
20007,tt6401142,Spiritus,2018,Horror,89,6.0,6.0,"<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...","<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...",One year after the mysterious death of Marjori...,123
20008,tt6402536,"Friends, Foes & Fireworks",2018,Drama,75,5.5,64.0,"<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...","<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...",Fiona has recently received a life-changing he...,84
20010,tt6406848,Dublin Oldschool,2018,Drama,95,6.0,729.0,"<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...","<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...","Join Jason, a wannabe DJ, on a drug-fuelled tr...",125
...,...,...,...,...,...,...,...,...,...,...,...
20483,tt6623390,The House of Violent Desire,2018,"Horror,Mystery,Thriller",117,3.3,323.0,"<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...","<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...","In a remote hill top mansion, a mysterious str...",123
20484,tt6624078,More Blood!,2018,"Documentary,Horror",\N,8.3,23.0,"<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...","<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...",Have you ever wondered why you root for people...,82
20488,tt6627630,Cycle,2018,Comedy,96,8.1,619.0,"<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...","<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...",Keshav a well respected astrologer in his vill...,84
20497,tt6632664,La chute de Sparte,2018,Drama,93,6.0,253.0,"<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...","<!DOCTYPE html><html lang=""en-US"" xmlns:og=""ht...","When you're young, sometimes it feels like the...",144


In [21]:
idx = 10
synopsis = df.iloc[idx]["synopsis"]
title = df.iloc[idx]["originalTitle"]

In [26]:
prompt=f"""
Instruction: Create a set of 2 question and answer pairs based on a provided movie description.
Description: How I failed, became the subject of my subject, made a self-reflexive film - or nearly all about religion, monsters and syncretism in Ghana? I started 2011 with filming about the West African concepts of monsters. But monsters don't seem to exist in Ghana and I was surprised that I went into the trap of universal ideas. But then I met David. And when he brought me to the shrine of Mamishie Rasta, he involved himself into a ritual. The conflict he faced after this was due to his Christian belief. This was the turning point of WE ARE THE OTHERS and I asked myself many questions as: What is African (Traditional) Religion at all? What happens when people combine different religions? And does monstrosity re-enters at some point?—Antje Akkermann
Q&A pairs:
[
 {{
   "question": "What is the movie We Are The Others about?",
   "answer":"The movie is a documentary about the filmmaker's experience in Ghana, where she discovered that there are no monsters as she had initially thought."
 }},
 {{
   "question": "Which location(s) are featured in the movie We Are The Others?",
   "answer":"Ghana"
 }},
  {{
   "question": "Which location(s) are featured in the movie We Are The Others?",
   "answer":"Ghana"
 }},
]
Instruction: Create a set of 2 question and answer pairs based on a provided movie description.
1. The movie name `{title.strip()}` should be mentioned in each question surrounded by double quotes
2. Output should be in a JSON list like [{{"question":"", "answer"}}, {{"question":"", "answer":""}},...].
Description: {synopsis.strip()}
Q&A pairs:
"""
print(prompt)


Instruction: Create a set of 2 question and answer pairs based on a provided movie description.
Description: How I failed, became the subject of my subject, made a self-reflexive film - or nearly all about religion, monsters and syncretism in Ghana? I started 2011 with filming about the West African concepts of monsters. But monsters don't seem to exist in Ghana and I was surprised that I went into the trap of universal ideas. But then I met David. And when he brought me to the shrine of Mamishie Rasta, he involved himself into a ritual. The conflict he faced after this was due to his Christian belief. This was the turning point of WE ARE THE OTHERS and I asked myself many questions as: What is African (Traditional) Religion at all? What happens when people combine different religions? And does monstrosity re-enters at some point?—Antje Akkermann
Q&A pairs:
[
 {
   "question": "What is the movie We Are The Others about?",
   "answer":"The movie is a documentary about the filmmaker's e

In [28]:
print(generate_questions_palm2(prompt))

[
  {
   "question": "What is Dublin Oldschool about?",
   "answer":"A wannabe DJ and his heroin addict brother"
 },
 {
   "question": "What drug does Daniel use?",
   "answer":"Heroin"
 },
]


In [29]:
import pandas as pd

In [30]:
df = pd.read_feather("./datasets/movie_datasets/imdb/movie_html_prompt_0.feather")

In [36]:
df.iloc[7]["palm2_op"]

'\n [\n {\n   "question": "What is the subject of the movie Enormous: The Gorge Story?",\n   "answer":"The Gorge, a Pollstar, Billboard, and ACM winning music venue"\n },\n {\n   "question": "What artists had legendary pasts at "The Gorge", the subject of the movie Enormous: The Gorge Story?",\n   "answer":"Dave Matthews, Jason Mraz, and Pearl Jam"\n },\n]'